### Library

In [4]:
from humpback import *

import pandas as pd
import numpy as np
from datetime import datetime
import ta

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

from backtesting import Backtest, Strategy

### TA Calculation Functions

#### SMA

In [5]:
def sma(df, window):
    sma = ta.trend.SMAIndicator(pd.Series(df), window=window).sma_indicator()
    return sma

#### RSI

In [6]:
def rsi(df, window=14):
    rsi = ta.momentum.RSIIndicator(pd.Series(df), window=window).rsi()
    return rsi

#### EMA

In [7]:
def ema(df, period=200):
    ema = ta.trend.EMAIndicator(pd.Series(df), window=window).ema_indicator()
    return ema

#### MACD

In [8]:
def macd(df):
    macd = ta.trend.MACD(pd.Series(df)).macd()
    return macd

#### Bollinger Bands

In [9]:
def signal_h(df):
    return ta.volatility.BollingerBands(pd.Series(df)).bollinger_hband()
def signal_l(df):
    return ta.volatility.BollingerBands(pd.Series(df)).bollinger_lband()

### other handy functions

In [10]:
def get_X(data):
    return data.filter(like='X')

In [11]:
def get_y(data):
    return data['Close']

In [12]:
def get_clean_Xy(data):
    data.dropna(inplace=True)
    X = get_X(data)
    y = get_y(data)
    return X, y

In [13]:
def featureGeneration(data):
    close = data.Close

    sma10 = sma(data.Close, 10)
    sma20 = sma(data.Close, 20)
    sma50 = sma(data.Close, 50)
    sma100 = sma(data.Close, 100)
    upper = signal_h(data.Close)
    lower = signal_l(data.Close)

    # Design matrix / independent features:

    # Price-derived features
    data['X_SMA10'] = (close - sma10) / close
    data['X_SMA20'] = (close - sma20) / close
    data['X_SMA50'] = (close - sma50) / close
    data['X_SMA100'] = (close - sma100) / close

    data['X_DELTA_SMA10'] = (sma10 - sma20) / close
    data['X_DELTA_SMA20'] = (sma20 - sma50) / close
    data['X_DELTA_SMA50'] = (sma50 - sma100) / close

    # Indicator features
    data['X_MOM'] = data.Close.pct_change(periods=2)
    data['X_BB_upper'] = (upper - close) / close
    data['X_BB_lower'] = (lower - close) / close
    data['X_BB_width'] = (upper - lower) / close

    # Some datetime features for good measure
    data['X_day'] = data.index.dayofweek
    data['X_hour'] = data.index.hour

    data = data.dropna().astype(float)
    return data

### Strategy Class

#### ML Strategy

In [ ]:
class MLstr(Strategy):
    train_split = .7 # 70% training 30% testing

    def init(self):
        
        pass

    def next(self):
        pass

### ML Model


## Main

### Binance API

In [15]:
client = connectBinanceAPI()

### Parameters

In [24]:
symbol = 'BTCUSDT'
interval = '1h'
window_LSTM = 60

start_str = int(datetime(2020,1,1,0,0).timestamp() * 1000)
end_str    = int(datetime(2023,12,31,0,0).timestamp() * 1000)

In [20]:
data = getBinanceData(client=client,
                      symbol=symbol,
                      interval=interval,
                      start_str=start_str,
                      end_str=end_str)

In [21]:
data_raw = data.copy()

In [25]:
data_forLSTM = create_previous_data_df(data[['Close']], window_LSTM)

### Fetch Historical Data

### Backtesting

#### Backtesting

#### Graph

## For Temp Ad Hoc Testing

In [ ]:
bt = Backtest(data, MLstr, cash = 1000000, commission = .0007, margin = .05)
bt.run()
bt.plot()

## Reference

Backtesting User Manual

https://kernc.github.io/backtesting.py/doc/examples/Quick%20Start%20User%20Guide.html